In [1]:
file1=r"F:\Happy data scientist\python\PROJECTS\project2\carvan_train.csv"
file2=r"F:\Happy data scientist\python\PROJECTS\project2\carvan_test.csv"

In [2]:
import pandas as pd
import numpy as np

In [3]:
ld_train=pd.read_csv(file1)
ld_test=pd.read_csv(file2)

In [4]:
ld_train.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86
0,33,1,3,2,8,0,5,1,3,7,...,0,0,0,1,0,0,0,0,0,0
1,37,1,2,2,8,1,4,1,4,6,...,0,0,0,1,0,0,0,0,0,0
2,37,1,2,2,8,0,4,2,4,3,...,0,0,0,1,0,0,0,0,0,0
3,9,1,3,3,3,2,3,2,4,5,...,0,0,0,1,0,0,0,0,0,0
4,40,1,4,2,10,1,4,1,4,7,...,0,0,0,1,0,0,0,0,0,0


In [5]:
ld_test["V86"]=np.nan


In [6]:
ld_train["data"]="train"
ld_test["data"]="test"

In [7]:
ld_train.shape

(5822, 87)

In [8]:
ld_test.shape

(4000, 87)

In [9]:
ld=pd.concat([ld_train,ld_test],axis=0)

In [10]:
ld.shape

(9822, 87)

In [11]:
ld["V44"].value_counts()

0    5903
2    3562
1     341
3      16
Name: V44, dtype: int64

In [12]:
cols=["V1","V4","V5","V6","V44"]

for col in cols:
    freq=ld[col].value_counts()
    freq=freq.index[freq>50][:-1]
    
    for f in freq:
        name=col+"_"+str(f)
        ld[name]=np.where(ld[col]==f,1,0).astype(int)
        
    del ld[col]
    

In [13]:
ld["V86"].isnull().sum()

4000

In [14]:
ld_train=ld.loc[ld["data"]=="train",]

In [15]:
ld_train.shape

(5822, 132)

In [16]:
ld_test=ld.loc[ld["data"]=="test",]

In [17]:
ld_test.shape

(4000, 132)

In [18]:
from sklearn.model_selection import train_test_split
ld_train_train,ld_train_val=train_test_split(ld_train,test_size=0.2,random_state=2)

In [19]:
ld_train_train.shape

(4657, 132)

In [20]:
del ld_test["data"]
del ld_train_train["data"]
del ld_train_val["data"]
del ld_test["V86"]

In [21]:
x_ld_train_train=ld_train_train.drop(["V86"],axis=1)
y_ld_train_train=ld_train_train["V86"]
x_ld_train_val=ld_train_val.drop(["V86"],axis=1)

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [23]:
logr=LogisticRegression(C=0.01,class_weight="balanced",penalty="l2")
logr.fit(x_ld_train_train,y_ld_train_train)
p=logr.predict_proba(x_ld_train_val)[:,1]

C:\Users\swadhin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [24]:
from sklearn.metrics import roc_auc_score
roc_auc_score(ld_train_val["V86"],p)

0.7326666666666668

In [25]:
logr.classes_

array([0., 1.])

In [26]:
params={"C":np.linspace(0.01,0.99,5)}

In [27]:
gridsearch=GridSearchCV(logr,param_grid=params,cv=5,scoring="roc_auc")

In [28]:
gridsearch.fit(x_ld_train_train,y_ld_train_train)

C:\Users\swadhin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\swadhin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\swadhin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\swadhin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\swadhin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=0.01, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([0.01 , 0.255, 0.5  , 0.745, 0.99 ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [29]:
gridsearch.best_estimator_

LogisticRegression(C=0.01, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [30]:
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [31]:
dtree=DecisionTreeClassifier()

In [32]:
params={"class_weight":["balanced",None],
       "criterion":["entropy","gini"],
       "max_depth":[8,10,15,20],
       "min_samples_split":[2,3,4,5,6],
       "min_samples_leaf":[1,2,3,4,5]}

In [33]:
rs=RandomizedSearchCV(dtree,param_distributions=params,cv=5,scoring="roc_auc",n_iter=10,n_jobs=-1)

In [34]:
rs.fit(x_ld_train_train,y_ld_train_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'class_weight': ['balanced', None], 'criterion': ['entropy', 'gini'], 'max_depth': [8, 10, 15, 20], 'min_samples_split': [2, 3, 4, 5, 6], 'min_samples_leaf': [1, 2, 3, 4, 5]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [35]:
p_dtree=rs.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_dtree)

0.6572844036697247

In [36]:
dtree.fit(x_ld_train_train,y_ld_train_train)
p_dtree=dtree.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_dtree)

0.5742079510703364

In [37]:
rf=RandomForestClassifier()


In [38]:
params={"class_weight":["balanced",None],
       "criterion":["entropy","gini"],
       "max_depth":[8,10,15,20],
       "min_samples_split":[2,3,4,5,6],
       "min_samples_leaf":[1,2,3,4,5],
       "n_estimators":[50,100,200,500],
       "max_features":[5,10,30,70]}

In [39]:
rs_rf=rs=RandomizedSearchCV(rf,param_distributions=params,cv=5,scoring="roc_auc",n_iter=10,n_jobs=-1)

In [40]:
rs_rf.fit(x_ld_train_train,y_ld_train_train)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'class_weight': ['balanced', None], 'criterion': ['entropy', 'gini'], 'max_depth': [8, 10, 15, 20], 'min_samples_split': [2, 3, 4, 5, 6], 'min_samples_leaf': [1, 2, 3, 4, 5], 'n_estimators': [50, 100, 200, 500], 'max_features': [5, 10, 30, 70]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [41]:
p_rf=rs_rf.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_rf)

0.7352844036697248

In [42]:
rf=RandomForestClassifier(bootstrap=False, class_weight='balanced',
            criterion='entropy', max_depth=8, max_features=30,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=3, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [43]:
rf.fit(x_ld_train_train,y_ld_train_train)
p_rf=rf.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_rf)

0.7467033639143731

In [44]:
fi=pd.DataFrame({"features":x_ld_train_train.columns,
                "importance":rf.feature_importances_})

In [45]:
r=fi.sort_values(["importance"],ascending=True)

In [46]:
params={"class_weight":["balanced",None],
       "criterion":["entropy","gini"],
       "max_depth":[8,10,15,20],
       "min_samples_split":[2,3,4,5,6],
       "min_samples_leaf":[1,2,3,4,5],
       "n_estimators":[50,100,200,500],
       "max_features":[5,10,30,70]}

In [47]:
et=ExtraTreesClassifier()
rs_et=RandomizedSearchCV(et,param_distributions=params,cv=5,scoring="roc_auc",n_iter=10,n_jobs=-1)

In [48]:
rs_et.fit(x_ld_train_train,y_ld_train_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'class_weight': ['balanced', None], 'criterion': ['entropy', 'gini'], 'max_depth': [8, 10, 15, 20], 'min_samples_split': [2, 3, 4, 5, 6], 'min_samples_leaf': [1, 2, 3, 4, 5], 'n_estimators': [50, 100, 200, 500], 'max_features': [5, 10, 30, 70]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [49]:
p_et=rs_et.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_et)

0.7320305810397553

In [59]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier

In [60]:
xg=XGBClassifier(objective="binary:logistic")

In [61]:
xg.fit(x_ld_train_train,y_ld_train_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [62]:
p_xg=xg.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_xg)

0.7515412844036697

In [54]:
cutoffs=np.linspace(0.01,0.99,99)
p_xg1=xg.predict_proba(x_ld_train_train)[:,1]

In [55]:
real=y_ld_train_train

In [56]:
f_beta=[]
for cutoff in cutoffs:
    predicted=(p_xg1>cutoff).astype(int)
    
    tp=((predicted==1) & (real==1)).sum()
    tn=((predicted==0) & (real==0)).sum()
    fp=((predicted==1) & (real==0)).sum()
    fn=((predicted==0) & (real==1)).sum()
    
    p=tp+fn
    n=tn+fp
    precision=tp/(tp+fp)
    recall=tp/p
    f_score=((1+4)*precision*recall)/((4*precision)+recall)
    
    f_beta.append(f_score)

C:\Users\swadhin\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  if sys.path[0] == '':


In [57]:
cutoffs[f_beta==max(f_beta)]

array([0.09])

In [58]:
predicted=(p_xg1>0.09).astype(int)

In [63]:
gb=GradientBoostingClassifier()

In [64]:
gb.fit(x_ld_train_train,y_ld_train_train)
p_gb=gb.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_gb)


0.746507645259939

In [65]:
params={"learning_rate":[0.001,0.01,0.2],
       "n_estimators":[50,100,200,300],
       "min_samples_split":[2,3,4,5],
       "max_depth":[3,4,5,6,8],
       "min_samples_leaf":[2,3,4,7],
       "max_features":[20,30,50,80],
       "subsample":[.5,.6,.7,.8,.9]}

In [66]:
rs_gb=RandomizedSearchCV(gb,param_distributions=params,cv=10,n_iter=10,n_jobs=-1,scoring="roc_auc")

In [67]:
rs_gb.fit(x_ld_train_train,y_ld_train_train)

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'learning_rate': [0.001, 0.01, 0.2], 'n_estimators': [50, 100, 200, 300], 'min_samples_split': [2, 3, 4, 5], 'max_depth': [3, 4, 5, 6, 8], 'min_samples_leaf': [2, 3, 4, 7], 'max_features': [20, 30, 50, 80], 'subsample': [0.5, 0.6, 0.7, 0.8, 0.9]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [68]:
p_rs_gb=rs_gb.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_rs_gb)

0.7420611620795106

In [69]:
rs_gb.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=3,
              max_features=30, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=7, min_samples_split=3,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=0.6, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [70]:
xg=XGBClassifier(objective="binary:logistic")
params={"learning_rate":[0.001,0.01,0.2],
       "n_estimators":[50,100,200,300],
       "min_samples_split":[2,3,4,5],
       "max_depth":[3,4,5,6,8],
       "min_samples_leaf":[2,3,4,7],
       "max_features":[20,30,50,80],
       "subsample":[.5,.6,.7,.8,.9],
       "colsample_bytree":[.5,.6,.7,.8,.9],
       "colsample_byleavel":[.5,.6,.7,.8,.9],
       "max_delta_step":[5,7,9,11]}

In [71]:
rs_xg=RandomizedSearchCV(xg,param_distributions=params,cv=10,n_iter=20,n_jobs=-1,scoring="roc_auc")

In [72]:
rs_xg.fit(x_ld_train_train,y_ld_train_train)
p_rs_xg=rs_xg.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_rs_xg)

0.7441896024464831

In [73]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

In [74]:
knn=KNeighborsClassifier(n_neighbors=10)
knn.fit(x_ld_train_train,y_ld_train_train)
p_knn=knn.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_knn)

0.5902446483180428

In [76]:
svm=SVC(probability=True)
svm.fit(x_ld_train_train,y_ld_train_train)
p_svm=svm.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_svm)

C:\Users\swadhin\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.628519877675841

In [77]:
nv=MultinomialNB()
nv.fit(x_ld_train_train,y_ld_train_train)
p_nv=nv.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_nv)

0.668091743119266

In [78]:
from sklearn.neural_network import MLPClassifier

In [80]:
nn=MLPClassifier()
params={"learning_rate":["constant","invscaling","adaptive"],
       "hidden_layer_sizes":[(20,25),(10,5,30),(40,25),(20,15)],
       "alpha":[.1,.2,.3],"activation":["relu","tanh","logistic"]}

rs_nn=RandomizedSearchCV(nn,param_distributions=params,cv=10,n_iter=20,scoring="roc_auc",n_jobs=-1)
rs_nn.fit(x_ld_train_train,y_ld_train_train)
p_rs_nn=rs_nn.predict_proba(x_ld_train_val)[:,1]
roc_auc_score(ld_train_val["V86"],p_rs_nn)

0.7269908256880733

In [86]:
x_ld_train=ld_train.drop(["V86"],axis=1)
y_ld_train=ld_train["V86"]

In [120]:
xgb=XGBClassifier(objective="binary:logistic",max_delta_step=5)
xgb.fit(x_ld_train,y_ld_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=5, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [121]:
p=xgb.predict_proba(x_ld_train)[:,1]

In [122]:
real=y_ld_train
f_beta=[]
for cutoff in cutoffs:
    predicted=(p>cutoff).astype(int)
    
    tp=((predicted==1) & (real==1)).sum()
    tn=((predicted==0) & (real==0)).sum()
    fp=((predicted==1) & (real==0)).sum()
    fn=((predicted==0) & (real==1)).sum()
    
    p=tp+fn
    n=tn+fp
    precision=tp/(tp+fp)
    recall=tp/p
    f_score=((1+4)*precision*recall)/((4*precision)+recall)
    
    f_beta.append(f_score)

In [123]:
cutoffs[f_beta==max(f_beta)]

array([0.01])

In [124]:
max(f_beta)

0.2562214695921072

In [137]:
rf=RandomForestClassifier(class_weight="balanced")
rf.fit(x_ld_train,y_ld_train)
p=rf.predict_proba(x_ld_train)[:,1]

C:\Users\swadhin\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [138]:
p=rf.predict_proba(ld_test)[:,1]

In [142]:
p=(p>0.01).astype(int)

In [144]:
pd.DataFrame(p).to_csv("harpreet_singh_P1_part2.csv",index=False)

In [135]:
s["predicted"].value_counts()

1    5822
Name: predicted, dtype: int64

In [136]:
pd.crosstab(s["real"],s["predicted"])

predicted,1
real,
0.0,5474
1.0,348
